In [ ]:
!pip install torchnet
!pip install utils

!pip install tqdm
!pip install nested_dict
!pip install numpy
!pip install torch
!pip install torchnet
!pip install torchvision

     |████████████████████████████████| 686kB 5.1MB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.4-cp36-none-any.whl size=29744 sha256=efc9539698d52c4a2bb28dd6aec3289a5dfa7bb5df9fcc4382d4c71ea5434f49
  Stored in directory: /root/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97fe39022946cf16b8e66ed754a6663400
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655250 sha256=abe0a714665aa26872f310eba975bf013a1a8e2bb623bc6d53c4485e169b7013
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=7a42885a056d130bbec689dfed21224678408ecf4958601be89bdfec83fc4335
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchnet visdom torchfile
  Created wheel for nested-dict: filename=nested_dict-1.61

In [ ]:
from nested_dict import nested_dict
from functools import partial
import torch
from torch.nn.init import kaiming_normal_
from torch.nn.parallel._functions import Broadcast
from torch.nn.parallel import scatter, parallel_apply, gather
import torch.nn.functional as F


def distillation(y, teacher_scores, labels, T, alpha):
    p = F.log_softmax(y/T, dim=1)
    q = F.softmax(teacher_scores/T, dim=1)
    l_kl = F.kl_div(p, q, size_average=False) * (T**2) / y.shape[0]
    l_ce = F.cross_entropy(y, labels)
    return l_kl * alpha + l_ce * (1. - alpha)


def at(x):
    return F.normalize(x.pow(2).mean(1).view(x.size(0), -1))


def at_loss(x, y):
    print("INSIDE AT_LOSS")
    print(x.shape)
    print(y.shape)
    print(at(x).shape)
    print(at(y).shape)
    return (at(x) - at(y)).pow(4).mean()


def cast(params, dtype='float'):
    if isinstance(params, dict):
        return {k: cast(v, dtype) for k,v in params.items()}
    else:
        return getattr(params.cuda() if torch.cuda.is_available() else params, dtype)()


def conv_params(ni, no, k=1):
    return kaiming_normal_(torch.Tensor(no, ni, k, k))


def linear_params(ni, no):
    return {'weight': kaiming_normal_(torch.Tensor(no, ni)), 'bias': torch.zeros(no)}


def bnparams(n):
    return {'weight': torch.rand(n),
            'bias': torch.zeros(n),
            'running_mean': torch.zeros(n),
            'running_var': torch.ones(n)}


def data_parallel(f, input, params, mode, device_ids, output_device=None):
    device_ids = list(device_ids)
    if output_device is None:
        output_device = device_ids[0]

    if len(device_ids) == 1:
        return f(input, params, mode)

    params_all = Broadcast.apply(device_ids, *params.values())
    params_replicas = [{k: params_all[i + j*len(params)] for i, k in enumerate(params.keys())}
                       for j in range(len(device_ids))]

    replicas = [partial(f, params=p, mode=mode)
                for p in params_replicas]
    inputs = scatter([input], device_ids)
    outputs = parallel_apply(replicas, inputs)
    return gather(outputs, output_device)


def flatten(params):
    return {'.'.join(k): v for k, v in nested_dict(params).items_flat() if v is not None}


def batch_norm(x, params, base, mode):
    return F.batch_norm(x, weight=params[base + '.weight'],
                        bias=params[base + '.bias'],
                        running_mean=params[base + '.running_mean'],
                        running_var=params[base + '.running_var'],
                        training=mode)


def print_tensor_dict(params):
    kmax = max(len(key) for key in params.keys())
    for i, (key, v) in enumerate(params.items()):
        print(str(i).ljust(5), key.ljust(kmax + 3), str(tuple(v.shape)).ljust(23), torch.typename(v), v.requires_grad)


def set_requires_grad_except_bn_(params):
    for k, v in params.items():
        if not k.endswith('running_mean') and not k.endswith('running_var'):
            v.requires_grad = True

In [ ]:
import argparse
import os
import json
import numpy as np
from tqdm import tqdm
import torch
from torch.optim import SGD
import torchvision.transforms as T
from torchvision import datasets
import torch.nn.functional as F
import torchnet as tnt
from torchnet.engine import Engine
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import utils
from easydict import EasyDict

cudnn.benchmark = True

In [ ]:
def model_parameters(args):
  epoch_step = json.loads(args.epoch_step)
  num_classes = args.classes
  
  def create_dataset(args, train):

    transform = T.Compose([
        T.ToTensor(),
        T.Normalize(np.array([125.3, 123.0, 113.9]) / 255.0,
                    np.array([63.0, 62.1, 66.7]) / 255.0),
    ])
    if args.channel==0:
      transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
      ])
    if train:
        transform = T.Compose([
            T.Pad(4, padding_mode='reflect'),
            T.RandomHorizontalFlip(),
            T.RandomCrop(32),
            transform
        ])
    return getattr(datasets, args.dataset)(args.dataroot, train=train, download=True, transform=transform)

  def create_iterator(mode):
          return DataLoader(create_dataset(args, mode), args.batch_size, shuffle=mode,
                            num_workers=args.nthread, pin_memory=torch.cuda.is_available())

  train_loader = create_iterator(True)
  test_loader = create_iterator(False)
  
  c=0
  if args.channel==0:
    c=1
  else:
    c=3
  
  def resnet(depth, width, num_classes):
      assert (depth - 4) % 6 == 0, 'depth should be 6n+4'
      n = (depth - 4) // 6
      widths = [int(v * width) for v in (16, 32, 64)]

      def gen_block_params(ni, no):
          return {
              'conv0': conv_params(ni, no, 3),
              'conv1': conv_params(no, no, 3),
              'bn0': bnparams(ni),
              'bn1': bnparams(no),
              'convdim': conv_params(ni, no, 1) if ni != no else None,
          }

      def gen_group_params(ni, no, count):
          return {'block%d' % i: gen_block_params(ni if i == 0 else no, no)
                  for i in range(count)}
      flat_params = cast(flatten({
          'conv0': conv_params(c, 16, 3),
          'group0': gen_group_params(16, widths[0], n),
          'group1': gen_group_params(widths[0], widths[1], n),
          'group2': gen_group_params(widths[1], widths[2], n),
          'bn': bnparams(widths[2]),
          'fc': linear_params(widths[2], num_classes),
      }))

      set_requires_grad_except_bn_(flat_params)

      def block(x, params, base, mode, stride):
          #print("INSIDE BLOCK")
          #print(x.shape)
          o1 = F.relu(batch_norm(x, params, base + '.bn0', mode), inplace=True)
          #print(o1.shape)
          y = F.conv2d(o1, params[base + '.conv0'], stride=stride, padding=1)
          #print(y.shape)
          o2 = F.relu(batch_norm(y, params, base + '.bn1', mode), inplace=True)
          #print(o2.shape)
          z = F.conv2d(o2, params[base + '.conv1'], stride=1, padding=1)
          #print(z.shape)
          if base + '.convdim' in params:
              #print("HERE in block")
              #print(z.shape)
              #print(F.conv2d(o1, params[base + '.convdim'], stride=stride).shape)
              return z + F.conv2d(o1, params[base + '.convdim'], stride=stride)
          else:
              return z + x

      def group(o, params, base, mode, stride):
          for i in range(n):
              o = block(o, params, f'{base}.block{i}', mode, stride if i == 0 else 1)
          return o

      def f(input, params, mode, base=''):
          #print("HERE F")
          #print(input.shape)
          x = F.conv2d(input, params[f'{base}conv0'], padding=1)
          #print("x")
          #print(x.shape)
          #print(x.shape)
          g0 = group(x, params, f'{base}group0', mode, 1)
          #print(g0.shape)
          g1 = group(g0, params, f'{base}group1', mode, 2)
          #print(g1.shape)
          g2 = group(g1, params, f'{base}group2', mode, 2)
          #print(g2.shape)
          o = F.relu(batch_norm(g2, params, f'{base}bn', mode))
          #print(o.shape)
          o = F.avg_pool2d(o, 8, 1, 0)
          #print(o.shape)
          o = o.view(o.size(0), -1)
          #print(o.shape)
          o = F.linear(o, params[f'{base}fc.weight'], params[f'{base}fc.bias'])
          #print(o.shape)
          return o, (g0, g1, g2)

      return f, flat_params




  f_s, params_s = resnet(args.depth, args.width, num_classes)
  if args.teacher_id:
          with open(os.path.join('logs', args.teacher_id, 'log.txt'), 'r') as ff:
              line = ff.readline()
              r = line.find('json_stats')
              info = json.loads(line[r + 12:])
          f_t = resnet(info['depth'], info['width'], num_classes)[0]
          model_data = torch.load(os.path.join('logs', args.teacher_id, 'model.pt7'))
          params_t = model_data['params']

          # merge teacher and student params
          params = {'student.' + k: v for k, v in params_s.items()}
          for k, v in params_t.items():
              params['teacher.' + k] = v.detach().requires_grad_(False)

          def f(inputs, params, mode):
              y_s, g_s = f_s(inputs, params, mode, 'student.')
              with torch.no_grad():
                  y_t, g_t = f_t(inputs, params, False, 'teacher.')
              return y_s, y_t, [at_loss(x, y) for x, y in zip(g_s, g_t)]
  else:
    f, params = f_s, params_s

  def create_optimizer(args, lr):
        print('creating optimizer with lr = ', lr)
        return SGD((v for v in params.values() if v.requires_grad), lr,
                   momentum=0.9, weight_decay=args.weight_decay)
  
  optimizer = create_optimizer(args, args.lr)
  
  
  epoch = 0
  
  if args.resume != '':
    state_dict = torch.load(args.resume)
    epoch = state_dict['epoch']
    params_tensors = state_dict['params']
    for k, v in params.items():
        v.data.copy_(params_tensors[k])
    optimizer.load_state_dict(state_dict['optimizer'])
    print("HERE")

  print('\nParameters:')
  print_tensor_dict(params)
  n_parameters = sum(p.numel() for p in list(params_s.values()))
  print('\nTotal number of parameters:', n_parameters)


  meter_loss = tnt.meter.AverageValueMeter()
  classacc = tnt.meter.ClassErrorMeter(accuracy=True)
  timer_train = tnt.meter.TimeMeter('s')
  timer_test = tnt.meter.TimeMeter('s')
  meters_at = [tnt.meter.AverageValueMeter() for i in range(3)]

  if not os.path.exists(args.save):
    os.mkdir(args.save)

  def h(sample):
        inputs = cast(sample[0], args.dtype).detach()
        targets = cast(sample[1], 'long')
        #print("INSIDE H")
        #print(inputs.shape)
        if inputs.shape[2]!=32:
          inputs = torch.nn.functional.pad(inputs, (3, 1, 3, 1))
        #print(inputs.shape)
        #print(targets.shape)
        if args.teacher_id != '':
            y_s, y_t, loss_groups = data_parallel(f, inputs, params, sample[2], range(1))
            print("LOSS _ GROUPS BEFORE")
            print(loss_groups)
            print(len(loss_groups))
            loss_groups = [v.sum() for v in loss_groups]
            print("LOSS _ GROUPS AFTER")
            print(loss_groups)
            [m.add(v.item()) for m, v in zip(meters_at, loss_groups)]
            return distillation(y_s, y_t, targets, args.temperature, args.alpha) \
                   + args.beta * sum(loss_groups), y_s
        else:
            y = data_parallel(f, inputs, params, sample[2], range(1))[0]
            return F.cross_entropy(y, targets), y

  def log(t, state):
      torch.save(dict(params={k: v.data for k, v in params.items()},
                      optimizer=state['optimizer'].state_dict(),
                      epoch=t['epoch']),
                  os.path.join(args.save, 'model.pt7'))
      z = vars(args).copy(); z.update(t)
      logname = os.path.join(args.save, 'log.txt')
      with open(logname, 'a') as f:
          f.write('json_stats: ' + json.dumps(z) + '\n')
      print(z)

  def on_sample(state):
      state['sample'].append(state['train'])

  def on_forward(state):
      classacc.add(state['output'].data, state['sample'][1])
      meter_loss.add(state['loss'].item())  

  def on_start(state):
        state['epoch'] = epoch

  def on_start_epoch(state):
      classacc.reset()
      meter_loss.reset()
      timer_train.reset()
      [meter.reset() for meter in meters_at]
      state['iterator'] = tqdm(train_loader)

      epoch = state['epoch'] + 1
      if epoch in epoch_step:
          lr = state['optimizer'].param_groups[0]['lr']
          state['optimizer'] = create_optimizer(args, lr * args.lr_decay_ratio)

  def on_end_epoch(state):
      train_loss = meter_loss.mean
      train_acc = classacc.value()
      train_time = timer_train.value()
      meter_loss.reset()
      classacc.reset()
      timer_test.reset()

      #print("IT CAME HERE")
      engine.test(h, test_loader)
      #print("FINISHED TESTIG")
      test_acc = classacc.value()[0]
      print(log({
          "train_loss": train_loss,
          "train_acc": train_acc[0],
          "test_loss": meter_loss.mean,
          "test_acc": test_acc,
          "epoch": state['epoch'],
          "num_classes": num_classes,
          "n_parameters": n_parameters,
          "train_time": train_time,
          "test_time": timer_test.value(),
          "at_losses": [m.value() for m in meters_at],
          }, state))
      print('==> id: %s (%d/%d), test_acc: \33[91m%.2f\033[0m' % \
                      (args.save, state['epoch'], args.epochs, test_acc))


  engine = Engine()
  engine.hooks['on_sample'] = on_sample
  engine.hooks['on_forward'] = on_forward
  engine.hooks['on_start_epoch'] = on_start_epoch
  engine.hooks['on_end_epoch'] = on_end_epoch
  engine.hooks['on_start'] = on_start
  engine.train(h, train_loader, args.epochs, optimizer)

  return engine

In [ ]:
teacher_args = EasyDict({
    "channel": 0,
    "classes": 10,
    "depth": 16,
    "width": 2,
    "dataset" : 'FashionMNIST',
    "dataroot" : '.',
    "dtype" : 'float',
    "nthread" : 0,
    "teacher_id" : '',
    "batch_size" : 128,
    "lr" : 0.1,
    "epochs" : 1,
    "weight_decay" : 0.0005,
    "epoch_step" : '[60, 120, 160]',
    "lr_decay_ratio" : 0.2,
    "resume" : '',
    "randomcrop_pad" : 4,
    "temperature" : 4,
    "alpha" : 0,
    "beta" : 0,
    "gpu_id" : '0',
    "save" : '/content/logs/resnet_52_2_teacher'
})

print('parsed options:', teacher_args)
teacher = model_parameters(teacher_args)



parsed options: {'channel': 0, 'classes': 10, 'depth': 16, 'width': 2, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 0, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 1, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/resnet_52_2_teacher'}


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


  0%|          | 0/469 [00:00<?, ?it/s]

creating optimizer with lr =  0.1

Parameters:
0     conv0                             (16, 1, 3, 3)           torch.cuda.FloatTensor True
1     group0.block0.conv0               (32, 16, 3, 3)          torch.cuda.FloatTensor True
2     group0.block0.conv1               (32, 32, 3, 3)          torch.cuda.FloatTensor True
3     group0.block0.bn0.weight          (16,)                   torch.cuda.FloatTensor True
4     group0.block0.bn0.bias            (16,)                   torch.cuda.FloatTensor True
5     group0.block0.bn0.running_mean    (16,)                   torch.cuda.FloatTensor False
6     group0.block0.bn0.running_var     (16,)                   torch.cuda.FloatTensor False
7     group0.block0.bn1.weight          (32,)                   torch.cuda.FloatTensor True
8     group0.block0.bn1.bias            (32,)                   torch.cuda.FloatTensor True
9     group0.block0.bn1.running_mean    (32,)                   torch.cuda.FloatTensor False
10    group0.block0.bn1.runnin

100%|██████████| 469/469 [00:41<00:00, 11.43it/s]


{'channel': 0, 'classes': 10, 'depth': 16, 'width': 2, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 0, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 1, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/resnet_52_2_teacher', 'train_loss': 0.5896742646373924, 'train_acc': 78.38000000000001, 'test_loss': 2.275277966185461, 'test_acc': 44.46, 'epoch': 1, 'num_classes': 10, 'n_parameters': 693210, 'train_time': 41.05589175224304, 'test_time': 2.5025694370269775, 'at_losses': [(nan, nan), (nan, nan), (nan, nan)]}
None
==> id: /content/logs/resnet_52_2_teacher (1/1), test_acc: 44.46


In [ ]:
student_trained_args = EasyDict({
    "channel": 0,
    "classes": 10,
    "depth": 10,
    "width": 1,
    "dataset" : 'FashionMNIST',
    "dataroot" : '.',
    "dtype" : 'float',
    "nthread" : 4,
    "teacher_id" : 'resnet_52_2_teacher',
    "batch_size" : 128,
    "lr" : 0.1,
    "epochs" : 3,
    "weight_decay" : 0.0005,
    "epoch_step" : '[60, 120, 160]',
    "lr_decay_ratio" : 0.2,
    "resume" : '',
    "randomcrop_pad" : 4,
    "temperature" : 4,
    "alpha" : 0,
    "beta" : 1000,
    "gpu_id" : '0',
    "save" : '/content/logs/at_52_2_16_1'
})

print('parsed options:', student_trained_args)
student_trained = model_parameters(student_trained_args)






  0%|          | 0/469 [00:00<?, ?it/s]

parsed options: {'channel': 0, 'classes': 10, 'depth': 10, 'width': 1, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 4, 'teacher_id': 'resnet_52_2_teacher', 'batch_size': 128, 'lr': 0.1, 'epochs': 3, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 1000, 'gpu_id': '0', 'save': '/content/logs/at_52_2_16_1'}
creating optimizer with lr =  0.1

Parameters:
0     student.conv0                             (16, 1, 3, 3)           torch.cuda.FloatTensor True
1     student.group0.block0.conv0               (16, 16, 3, 3)          torch.cuda.FloatTensor True
2     student.group0.block0.conv1               (16, 16, 3, 3)          torch.cuda.FloatTensor True
3     student.group0.block0.bn0.weight          (16,)                   torch.cuda.FloatTensor True
4     student.group0.block0.bn0.bias            (16,)                   torch.cuda.FloatTensor True
5     student.g

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



  0%|          | 1/469 [00:00<02:34,  3.03it/s]

INSIDE AT_LOSS
torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 32, 32])
torch.Size([128, 1024])
torch.Size([128, 1024])
INSIDE AT_LOSS
torch.Size([128, 32, 16, 16])
torch.Size([128, 64, 16, 16])
torch.Size([128, 256])
torch.Size([128, 256])
INSIDE AT_LOSS
torch.Size([128, 64, 8, 8])
torch.Size([128, 128, 8, 8])
torch.Size([128, 64])
torch.Size([128, 64])
LOSS _ GROUPS BEFORE
[tensor(6.1291e-07, device='cuda:0', grad_fn=<MeanBackward0>), tensor(1.2115e-05, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)]
3
LOSS _ GROUPS AFTER
[tensor(6.1291e-07, device='cuda:0', grad_fn=<SumBackward0>), tensor(1.2115e-05, device='cuda:0', grad_fn=<SumBackward0>), tensor(0.0001, device='cuda:0', grad_fn=<SumBackward0>)]
INSIDE AT_LOSS
torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 32, 32])
torch.Size([128, 1024])
torch.Size([128, 1024])
INSIDE AT_LOSS
torch.Size([128, 32, 16, 16])
torch.Size([128, 64, 16, 16])
torch.Size([128, 256])
torch.Size([1




  1%|          | 4/469 [00:00<01:54,  4.08it/s]


  1%|▏         | 7/469 [00:00<01:25,  5.41it/s]

...




 32%|███▏      | 148/469 [00:06<00:13, 23.78it/s]

[tensor(1.6422e-07, device='cuda:0', grad_fn=<MeanBackward0>), tensor(1.2813e-06, device='cuda:0', grad_fn=<MeanBackward0>), tensor(5.2561e-06, device='cuda:0', grad_fn=<MeanBackward0>)]
3
LOSS _ GROUPS AFTER
[tensor(1.6422e-07, device='cuda:0', grad_fn=<SumBackward0>), tensor(1.2813e-06, device='cuda:0', grad_fn=<SumBackward0>), tensor(5.2561e-06, device='cuda:0', grad_fn=<SumBackward0>)]
INSIDE AT_LOSS
torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 32, 32])
torch.Size([128, 1024])
torch.Size([128, 1024])
INSIDE AT_LOSS
torch.Size([128, 32, 16, 16])
torch.Size([128, 64, 16, 16])
torch.Size([128, 256])
torch.Size([128, 256])
INSIDE AT_LOSS
torch.Size([128, 64, 8, 8])
torch.Size([128, 128, 8, 8])
torch.Size([128, 64])
torch.Size([128, 64])
LOSS _ GROUPS BEFORE
[tensor(1.5203e-07, device='cuda:0', grad_fn=<MeanBackward0>), tensor(1.1031e-06, device='cuda:0', grad_fn=<MeanBackward0>), tensor(5.0023e-06, device='cuda:0', grad_fn=<MeanBackward0>)]
3
LOSS _ GROUPS AFTER
[tensor(1.5203e-0




 32%|███▏      | 151/469 [00:06<00:13, 23.43it/s]


 33%|███▎      | 154/469 [00:06<00:13, 23.56it/s]

INSIDE AT_LOSS
torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 32, 32])
torch.Size([128, 1024])
torch.Size([128, 1024])
INSIDE AT_LOSS
torch.Size([128, 32, 16, 16])
torch.Size([128, 64, 16, 16])
torch.Size([128, 256])
torch.Size([128, 256])
INSIDE AT_LOSS
torch.Size([128, 64, 8, 8])
torch.Size([128, 128, 8, 8])
torch.Size([128, 64])
torch.Size([128, 64])
LOSS _ GROUPS BEFORE
[tensor(1.6050e-07, device='cuda:0', grad_fn=<MeanBackward0>), tensor(1.8373e-06, device='cuda:0', grad_fn=<MeanBackward0>), tensor(5.9982e-06, device='cuda:0', grad_fn=<MeanBackward0>)]
3
LOSS _ GROUPS AFTER
[tensor(1.6050e-07, device='cuda:0', grad_fn=<SumBackward0>), tensor(1.8373e-06, device='cuda:0', grad_fn=<SumBackward0>), tensor(5.9982e-06, device='cuda:0', grad_fn=<SumBackward0>)]
INSIDE AT_LOSS
torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 32, 32])
torch.Size([128, 1024])
torch.Size([128, 1024])
INSIDE AT_LOSS
torch.Size([128, 32, 16, 16])
torch.Size([128, 64, 16, 16])
torch.Size([128, 256])
torch

KeyboardInterrupt: ignored

In [ ]:
student_args = EasyDict({
    "channel": 0,
    "classes": 10,
    "depth": 10,
    "width": 1,
    "dataset" : 'FashionMNIST',
    "dataroot" : '.',
    "dtype" : 'float',
    "nthread" : 4,
    "teacher_id" : '',
    "batch_size" : 128,
    "lr" : 0.1,
    "epochs" : 10,
    "weight_decay" : 0.0005,
    "epoch_step" : '[60, 120, 160]',
    "lr_decay_ratio" : 0.2,
    "resume" : '',
    "randomcrop_pad" : 4,
    "temperature" : 4,
    "alpha" : 0,
    "beta" : 0,
    "gpu_id" : '0',
    "save" : '/content/logs/at_10_1'
})

print('parsed options:', student_args)
student = model_parameters(student_args)




  0%|          | 0/469 [00:00<?, ?it/s]

parsed options: {'channel': 0, 'classes': 10, 'depth': 10, 'width': 1, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 4, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 10, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/at_10_1'}
creating optimizer with lr =  0.1

Parameters:
0     conv0                             (16, 1, 3, 3)           torch.cuda.FloatTensor True
1     group0.block0.conv0               (16, 16, 3, 3)          torch.cuda.FloatTensor True
2     group0.block0.conv1               (16, 16, 3, 3)          torch.cuda.FloatTensor True
3     group0.block0.bn0.weight          (16,)                   torch.cuda.FloatTensor True
4     group0.block0.bn0.bias            (16,)                   torch.cuda.FloatTensor True
5     group0.block0.bn0.running_mean    (16,)                   torch.cuda.FloatT

  0%|          | 0/469 [00:00<?, ?it/s]

{'channel': 0, 'classes': 10, 'depth': 10, 'width': 1, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 4, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 10, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/at_10_1', 'train_loss': 0.7520362459011934, 'train_acc': 72.68166666666667, 'test_loss': 1.4189174643045739, 'test_acc': 60.11, 'epoch': 1, 'num_classes': 10, 'n_parameters': 78042, 'train_time': 13.536037921905518, 'test_time': 1.1514101028442383, 'at_losses': [(nan, nan), (nan, nan), (nan, nan)]}
None
==> id: /content/logs/at_10_1 (1/10), test_acc: 60.11


  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/469 [00:00<?, ?it/s]

{'channel': 0, 'classes': 10, 'depth': 10, 'width': 1, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 4, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 10, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/at_10_1', 'train_loss': 0.3109663669297944, 'train_acc': 88.64, 'test_loss': 1.0020114651209187, 'test_acc': 68.48, 'epoch': 7, 'num_classes': 10, 'n_parameters': 78042, 'train_time': 13.490202188491821, 'test_time': 1.1207621097564697, 'at_losses': [(nan, nan), (nan, nan), (nan, nan)]}
None
==> id: /content/logs/at_10_1 (7/10), test_acc: 68.48


  0%|          | 0/469 [00:00<?, ?it/s]

{'channel': 0, 'classes': 10, 'depth': 10, 'width': 1, 'dataset': 'FashionMNIST', 'dataroot': '.', 'dtype': 'float', 'nthread': 4, 'teacher_id': '', 'batch_size': 128, 'lr': 0.1, 'epochs': 10, 'weight_decay': 0.0005, 'epoch_step': '[60, 120, 160]', 'lr_decay_ratio': 0.2, 'resume': '', 'randomcrop_pad': 4, 'temperature': 4, 'alpha': 0, 'beta': 0, 'gpu_id': '0', 'save': '/content/logs/at_10_1', 'train_loss': 0.304773916472504, 'train_acc': 89.15333333333334, 'test_loss': 1.5166614870481854, 'test_acc': 60.28, 'epoch': 8, 'num_classes': 10, 'n_parameters': 78042, 'train_time': 13.576308965682983, 'test_time': 1.1312522888183594, 'at_losses': [(nan, nan), (nan, nan), (nan, nan)]}
None
==> id: /content/logs/at_10_1 (8/10), test_acc: 60.28


 19%|█▉        | 88/469 [00:02<00:11, 33.11it/s]